# **Proyecto Final Inteligencia Artificial**

### Autores: **Angel David Piñeros Sierra**, **Camilo Andrés Roncancio Toca**, **Kelly Johana Solano Calderón**
### Presentado a: **Darwin Eduardo Martinez Riaño**


---

## **Modelo de segmentación de imágenes para la localización de lesiones asociadas al cáncer de piel**

### *Glosario*

### *(A) Descripción de la problemática*

### *(B) Objetivo*

### *(C) Importación y organización de datos*


In [3]:
import os
import zipfile
import pandas as pd
import subprocess

%pip install kaggle

dataset_url = "surajghuwalewala/ham1000-segmentation-and-classification"
ruta_descarga = "ham1000_data"
os.makedirs(ruta_descarga, exist_ok=True)
subprocess.run([
    "kaggle", "datasets", "download",
    "-d", dataset_url,
    "-p", ruta_descarga,
    "--unzip"
], check=True)

print("Contenido de la carpeta:", os.listdir(ruta_descarga))

images_dir = os.path.join(ruta_descarga, "images")
masks_dir = os.path.join(ruta_descarga, "masks")

if os.path.isdir(images_dir):
    print("Número de imágenes:", len(os.listdir(images_dir)))
else:
    print("No encontré la carpeta 'images'.")

if os.path.isdir(masks_dir):
    print("Número de máscaras:", len(os.listdir(masks_dir)))
else:
    print("No encontré la carpeta 'masks'.")

Dataset URL: https://www.kaggle.com/datasets/surajghuwalewala/ham1000-segmentation-and-classification
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)


100%|██████████| 2.59G/2.59G [00:48<00:00, 57.9MB/s]



Contenido de la carpeta: ['GroundTruth.csv', 'masks', 'images']
Número de imágenes: 10017
Número de máscaras: 10015
